---
layout: post  
---

Objective: Evaluate how the number of unique kmers observed increases relative to depth of coverage and error rate

I know that when resequencing a genome with a non-zero error rate and high redundancy (depth of coverage), eventually the number or erroneous kmers will dominate the number of true kmers that actually exist in the sample.
In order to avoid using an unnecessarily large amount of memory storing information for kmers that don't exist, error correction should happen at a k-size where some, but not all, kmers are erroneous.
Said another way, the idea is to do an iterative search (at a larger k) and bound (by correcting the erroneous ones)

In [57]:
import Pkg
pkgs = [
    "BioSequences",
    "StatsPlots",
    "Random",
    "StatsBase",
    "PlotlyJS",
    "Primes",
    "BioSymbols",
    "DataFrames",
    "ProgressMeter",
    "Colors",
    "Measures"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

StatsPlots.plotlyjs()

  Resolving package versions...
Updating `~/.julia/environments/v1.5/Project.toml`
  [442fdcdd] + Measures v0.3.1
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


Plots.PlotlyJSBackend()

In [2]:
function observe(sequence; error_rate = error_rate, alphabet = BioSymbols.ACGT)
    new_seq = Vector{eltype(sequence)}()
    for character in sequence
        if rand() > error_rate
            # match
            push!(new_seq, character)
        else
            error_type = rand(1:3)
            if error_type == 1
                # mismatch
                push!(new_seq, rand(setdiff(alphabet, character)))
            elseif error_type == 2
                # insertion
                push!(new_seq, rand(alphabet))
                push!(new_seq, character)
            else
                # deletion
                continue
            end
        end
    end
    return_seq = BioSequences.LongDNASeq(new_seq)
    if rand(Bool)
        return_seq = BioSequences.reverse_complement!(return_seq)
    end
    return return_seq
end

observe (generic function with 1 method)

In [ ]:
results = 
DataFrames.DataFrame(
    k = [],
    error_rate = [],
    sequence_length = [],
    coverage = [],
    unique_kmers = [])

iteration = 1
ProgressMeter.@showprogress for k in Primes.primes(3, 31)
    KMER_TYPE = BioSequences.DNAMer{k}
    for sequence_length in [10^i for i in 2:4]
        for error_rate in [0.0, 0.001, 0.01, 0.1]
            for coverage in [10^i for i in 1:3]
                sequence = BioSequences.randdnaseq(Random.seed!(iteration), sequence_length)
                kmers = Set{KMER_TYPE}()
                for i in 1:coverage
                    observed_sequence = observe(sequence, error_rate = error_rate) 
                    for kmer_set in BioSequences.each(KMER_TYPE, observed_sequence)
                        push!(kmers, BioSequences.canonical(kmer_set.fw))
                    end
                end
                unique_kmers = length(kmers)
                result = (
                        k = k,
                        error_rate = error_rate,
                        sequence_length = sequence_length,
                        coverage = coverage,
                        unique_kmers = unique_kmers
                        )
                push!(results, result)
                iteration += 1
            end
        end
    end
end

In [104]:
error_to_linestyles = Dict(
    a => b for (a, b) in zip(sort(unique(results[!, "error_rate"])), [:solid, :dash, :dot, :dashdot])
)

Dict{Float64,Symbol} with 4 entries:
  0.0   => :solid
  0.01  => :dot
  0.1   => :dashdot
  0.001 => :dash

In [40]:
coverage_to_markershape = Dict(
    a => b for (a, b) in zip(sort(unique(results[!, "coverage"])), [:circle, :rect, :cross])
)

Dict{Int64,Symbol} with 3 entries:
  100  => :rect
  10   => :circle
  1000 => :cross

In [126]:
for sequence_length in sort(unique(results[!, "sequence_length"]))
    p = StatsPlots.plot(
        title = "Sequence Length : $(sequence_length)",
        xlabel = "k",
        ylabel = "# unique observed kmers",
        legend = :outertopright,
        size = (700, 500),
        leftmargin = 5(Measures.mm)
    )

    for error_rate in sort(unique(results[!, "error_rate"]))
        for coverage in sort(unique(results[!, "coverage"]))
            indices = (
                (results[!, "sequence_length"] .== sequence_length) .&
                (results[!, "error_rate"] .== error_rate) .&
                (results[!, "coverage"] .== coverage))
            xs = results[indices, "k"]
            ys = results[indices, "unique_kmers"]

            p = StatsPlots.plot!(p,
                xs,
                ys,
                xticks = (xs, string.(xs)),
                linestyle = error_to_linestyles[error_rate],
                markershape = coverage_to_markershape[coverage],
                label = "error_rate: $(rpad(error_rate, 5, '0')) | coverage:$(coverage)"
            )
        end
    end
    # displays in notebook
    display(p)
    
    # displays on website
    filename = "/assets/images/2020-12-19-observed-kmers-vs-error-rate-$(sequence_length).html"
    StatsPlots.savefig(p, dirname(pwd()) * filename)
    display("text/html", "![]($filename)")
end

<!DOCTYPE html>
 
 
 Plots.jl

![](/assets/images/2020-12-19-observed-kmers-vs-error-rate-100.html)

<!DOCTYPE html>
 
 
 Plots.jl

![](/assets/images/2020-12-19-observed-kmers-vs-error-rate-1000.html)

<!DOCTYPE html>
 
 
 Plots.jl

![](/assets/images/2020-12-19-observed-kmers-vs-error-rate-10000.html)

This is very interesting to me!
The top two lines in each of these plots have "cross" marker shapes, indicating 1000x coverage.
This means that for these results, a 10x increase in coverage contributes more to overall noise than a 10x increase in error rate.
These results also point to the divergence zone for erroneous noise out-pacing signal occurs at the k=7-11 mark.


Because we can't see much beyond detail in the lower plots because of how much the `error_rate: 0.100 | coverage: 1000` results dominate the rest, we'll re-plot these with a log2(y) axis

In [127]:
for sequence_length in sort(unique(results[!, "sequence_length"]))
    p = StatsPlots.plot(
        title = "Sequence Length : $(sequence_length)",
        xlabel = "k",
        ylabel = "log2(# unique observed kmers)",
        legend = :outertopright,
        size = (700, 500),
        leftmargin = 5(Measures.mm)
    )

    for error_rate in sort(unique(results[!, "error_rate"]))
        for coverage in sort(unique(results[!, "coverage"]))
            indices = (
                (results[!, "sequence_length"] .== sequence_length) .&
                (results[!, "error_rate"] .== error_rate) .&
                (results[!, "coverage"] .== coverage))
            xs = results[indices, "k"]
            ys = results[indices, "unique_kmers"]

            p = StatsPlots.plot!(p,
                xs,
                log2.(ys),
                xticks = (xs, string.(xs)),
                linestyle = error_to_linestyles[error_rate],
                markershape = coverage_to_markershape[coverage],
                label = "error_rate: $(rpad(error_rate, 5, '0')) | coverage:$(coverage)"
            )
        end
    end
    # displays in notebook
    display(p)
    
    # displays on website
    filename = "/assets/images/2020-12-19-observed-kmers-vs-error-rate-$(sequence_length).log-transform.html"
    StatsPlots.savefig(p, dirname(pwd()) * filename)
    display("text/html", "![]($filename)")
end

<!DOCTYPE html>
 
 
 Plots.jl

![](/assets/images/2020-12-19-observed-kmers-vs-error-rate-100.log-transform.html)

<!DOCTYPE html>
 
 
 Plots.jl

![](/assets/images/2020-12-19-observed-kmers-vs-error-rate-1000.log-transform.html)

<!DOCTYPE html>
 
 
 Plots.jl

![](/assets/images/2020-12-19-observed-kmers-vs-error-rate-10000.log-transform.html)

In the above plots, we can divergence points for each of the sequence lengths:
- 100bp sequence
    - 3
- 1,000bp
    - 5
- 10,000bp
    - 7

I don't know if this pattern will hold in in practice on real data (these sequencs are simulated without any attention paid to real genetic grammar), but these breakpoints appear to follow a log4(sequence_length) pattern:

In [128]:
Int(round(log(4, 100)))

3

In [114]:
Int(round(log(4, 1000)))

5

In [115]:
Int(round(log(4, 10000)))

7